In [52]:
ls

 O volume na unidade C é Acer
 O Número de Série do Volume é 5620-EA58

 Pasta de C:\Users

28/04/2018  17:00    <DIR>          .
28/04/2018  17:00    <DIR>          ..
09/04/2018  02:21    <DIR>          defaultuser0
28/04/2018  09:38    <DIR>          João Gabriel
09/04/2018  01:59    <DIR>          Public
07/05/2018  16:11    <DIR>          trab
               0 arquivo(s)              0 bytes
               6 pasta(s)   903.820.468.224 bytes disponíveis


In [41]:
#importando os pacotes nescessarios
import numpy as np
import glob
from astropy.io import fits
# definindo o endereco x onde se encontram sua pasta, no caso xo2b, em que dentro dela havera as pastas bias,science e flat
x = './trab/desktop/xo2b'
# funcao bias que retorna o masterbias dos bias da pasta
def bias(y):
    '''
    This function recives a y variable that if == 1, it creates a fits file at the x adress. The fits is a masterbias of the paste bias.
    If y is diferent from 1, it returns only an array of the masterbias.
    '''
    #criando uma lista em que cada elemento fica como uma string do endereco de cada bias
    a = glob.glob(x+"/bias/*.fits")
    masterbias = []
    # laco for que ira jogar dentro da lista vazia masterbias os arrays de cada bias
    for i in range(len(a)):
        # usando o fits.getdata para retirar uma array de cada imagem bias
        img = fits.getdata(a[i])
        # transformando os elementos da array em float64
        img = img.astype(np.float64)
        # jogando essa array dentro da lista masterbias
        masterbias.append(img)
    # teste estatistico que mostra se o masterbias esta com uma media razoavel, menor que 20, ou nao.
    if np.mean(np.median(masterbias,axis=0)) > 20:
        return "Error: The masterbias mean is too higher(the mean is higher then 20)"
    # verificando a variavel y para retornar uma array da mediana da lista masterbias, caso y == 1 
    if y == 1:
        return np.median(masterbias,axis=0)
    # caso contrario, ele cria uma imagem fits masterbias no endereco x definido anteriormente
    else:
        outfile = x + '/master_bias.fits'
        hdu = fits.PrimaryHDU() #criando o HDU 
        hdu.data = np.median(masterbias,axis=0)
        hdu.writeto(outfile)

In [30]:
help(bias)


Help on function bias in module __main__:

bias(y)
    This function recives a y variable that if == 1, it creates a fits file at the x adress. The fits is a masterbias of the paste bias.
    If y is diferent from 1, it returns only an array of the masterbias.



In [23]:
# funcao flat que retorna o masterflat dos flats da pasta flat, ja reduzindo do masterbias
def flat(d):
    '''
    This function recives a d variable that if == 1, it creates a fits file reduced from bias at the x adress. 
    The fits is a masterflatbias of the paste flat.
    If d is diferent from 1, it returns only an array of the masterflatbias.
    '''
    #criando uma lista em que cada elemento fica como uma string do endereco de cada flat
    b = glob.glob(x+"/flat/*.fits")
    masterflat = []
    masterflatbias = []
    # laco for que ira jogar dentro da lista vazia masterflat os arrays de cada flat, e depois ira jogar dentro
    for i in range(len(b)):
        img = fits.getdata(b[i])
        img = img.astype(np.float64)
        masterflat.append(img)
        masterflatbias.append((masterflat[i] - bias(1))/np.median(masterflat[i]))
    if np.mean(np.median(masterflatbias,axis=0)) > 1.1:
            return "Error: The masterflat mean is too higher after normalization(the mean is higher then 1.1)"
    if np.mean(np.median(masterflatbias,axis=0)) < 0.9:
            return "Error: The masterflat mean is too lower after normalization(the mean is lower then 0.9)"
    if d == 1:
        return np.median(masterflatbias,axis=0)
    else:
        outfile = x + '/master_flat_bias.fits'
        hdu = fits.PrimaryHDU() #criando o HDU 
        hdu.data = np.median(masterflatbias,axis=0)
        hdu.writeto(outfile)

In [24]:
flat(1)

array([[1.08630775, 0.31210575, 0.31779009, ..., 0.33572358, 1.25562973,
        1.25602728],
       [0.90544953, 0.22328618, 0.22743632, ..., 0.22414036, 1.1093883 ,
        1.10972952],
       [0.92033337, 0.25336732, 0.25413839, ..., 0.25696047, 1.19559382,
        1.19594568],
       ...,
       [0.97763469, 0.26038239, 0.26106341, ..., 0.26731792, 1.17567192,
        1.17594639],
       [0.9544005 , 0.26088381, 0.26123909, ..., 0.27186694, 1.15585305,
        1.15611839],
       [0.90459092, 0.26285855, 0.26133482, ..., 0.2685868 , 1.07266922,
        1.07291897]])

In [59]:
def science():
    c = glob.glob(x+'/science/*.fits')
    for i in range(len(c)):
        img = fits.getdata(c[i])
        img = img.astype(np.float64)
        sci_clean = (img-bias(1))/flat(1)
        outfile = c[i][:-5] + "_clean.fits"
        hdu = fits.PrimaryHDU()
        hdu.data = sci_clean
        hdu.writeto(outfile)

In [50]:
def test():
    img = fits.getdata(x+'/science/xo2b.0002.fits')
    img = img.astype(np.float64)
    img2 = fits.getdata(x+'./science/xo2b.0002_clean.fits')
    img2 = img2.astype(np.float64)
    residual = (img - img2)
    outfile = x + '/science/xo2b.0002_residual.fits'
    hdu = fits.PrimaryHDU()
    hdu.data = residual
    hdu.writeto(outfile)
    
    

In [51]:
test()

In [34]:
x

'./trab/desktop/xo2b'